In [1]:
! pwd

/mnt/localdata/geng/lib/lmtc-eurlex57k


In [1]:
import json
import re
import time
import tempfile
import glob
import tqdm

import numpy as np
from copy import deepcopy
from collections import Counter
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import Sequence
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score, precision_score, recall_score


from json_loader import JSONLoader
from vectorizer import W2VVectorizer, ELMoVectorizer, BERTVectorizer
from data import DATA_SET_DIR, MODELS_DIR
from configuration import Configuration
from metrics import mean_recall_k, mean_precision_k, mean_ndcg_score, mean_rprecision_k
from neural_networks.lmtc_networks.document_classification import DocumentClassification
from neural_networks.lmtc_networks.label_driven_classification import LabelDrivenClassification


Using TensorFlow backend.


In [2]:
Configuration.configure()

In [3]:
import lmtc

In [4]:
lmtc=lmtc.LMTC()

Load labels' data
-------------------
100%|██████████| 12000/12000 [00:23<00:00, 501.91it/s] 
Labels shape:    4271
Frequent labels: 739
Few labels:      3369
Zero labels:     163


In [5]:
network = LabelDrivenClassification(lmtc.label_terms_ids)

In [6]:
lmtc.label_terms_ids

array([[11820,     0,     0, ...,     0,     0,     0],
       [  485,   310,     0, ...,     0,     0,     0],
       [  485,   324,  3908, ...,     0,     0,     0],
       ...,
       [ 1271,  2998,     0, ...,     0,     0,     0],
       [  723,     6,  1070, ...,     0,     0,     0],
       [  172,   566,   176, ...,     0,     0,     0]], dtype=int32)

In [32]:
import pickle
import numpy as np

with open("/mnt/localdata/geng/model/downstream/multiLabelClassification/legalBert/prediction.pickle", "rb") as f:
    pred_labels,true_labels=pickle.load(f)

In [6]:
from sklearn.metrics import classification_report

In [33]:
np.amax(pred_labels)

0.07959438

In [47]:
from math import exp

In [48]:
def sigmoid(x):
    1/(1+exp(-x))

In [55]:
sigmoid = lambda x: 1/(1+exp(-x))
vfunc = np.vectorize(sigmoid)
vfunc(pred_labels)

array([[0.50015305, 0.50024703, 0.50016784, ..., 0.50010097, 0.50020254,
        0.50016558],
       [0.50015305, 0.50024703, 0.50016789, ..., 0.50010098, 0.50020257,
        0.50016558],
       [0.50015306, 0.50024703, 0.50016786, ..., 0.50010098, 0.50020257,
        0.50016558],
       ...,
       [0.50015304, 0.50024702, 0.50016786, ..., 0.50010098, 0.50020256,
        0.50016558],
       [0.50015305, 0.50024702, 0.50016787, ..., 0.50010097, 0.50020258,
        0.50016557],
       [0.50015304, 0.50024703, 0.50016786, ..., 0.50010097, 0.50020256,
        0.50016557]])

In [49]:
pred_labels=np.array(pred_labels)

In [52]:
x=np.array(map(sigmoid, pred_labels))

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [42]:
np.max(np.sum(pred_labels,axis=1))

6.6422567

In [27]:
threshold=np.median(pred_labels)

y_pred=(pred_labels>threshold).astype(int)

np.sum(y_pred)/len(y_pred)

In [13]:
pred_labels=np.array(pred_labels)

In [16]:
sum(sum(pred_labels>0.5))

0

In [8]:
classification_report(y_true, y_pred)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [20]:
pred_labels=np.array(pred_labels)
true_labels=np.array(true_labels)

In [10]:
test_documents = lmtc.load_dataset('test')


100%|██████████| 100/100 [00:15<00:00,  6.42it/s]


In [11]:
limit = len(test_documents) % Configuration['model']['batch_size'] if Configuration['model']['architecture'] == 'BERT' else 0
test_samples, test_tags = lmtc.process_dataset(test_documents if not limit else test_documents[:-limit])
test_samples, test_targets = lmtc.encode_dataset(test_samples, test_tags)

In [12]:

test_samples, test_targets =test_samples[:100], test_targets[:100]

In [13]:
network.compile(n_hidden_layers=Configuration['model']['n_hidden_layers'],
                hidden_units_size=Configuration['model']['hidden_units_size'],
                dropout_rate=Configuration['model']['dropout_rate'],
                word_dropout_rate=Configuration['model']['word_dropout_rate'],
                lr=Configuration['model']['lr'])


loading projection weights from /mnt/localdata/geng/lib/lmtc-eurlex57k/data/vectors/law2vec.200d.txt
loaded (169439, 200) matrix from /mnt/localdata/geng/lib/lmtc-eurlex57k/data/vectors/law2vec.200d.txt


In [14]:
predictions=lmtc.predict(network,test_samples)

In [21]:
true_labels.shape

(6000, 4193)

In [15]:
predictions.shape

(100, 4271)

In [3]:
!pwd

/mnt/localdata/geng/lib/lmtc-eurlex57k


In [22]:

lmtc.calculate_performance(true_labels,pred_labels)

Overall
----------------------------------------------------
/mnt/localdata/geng/anaconda3/envs/lmtc_/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
micro    - Precision: 0.0000   Recall: 0.0000   F1: 0.0000
/mnt/localdata/geng/anaconda3/envs/lmtc_/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/localdata/geng/anaconda3/envs/lmtc_/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` par

In [13]:
documents = lmtc.load_dataset('test')

AttributeError: module 'lmtc' has no attribute 'load_dataset'

In [15]:
len(documents)

6000

In [19]:
lmtc.load_label_descriptions()

Load labels' data
-------------------
100%|██████████| 12000/12000 [00:09<00:00, 1242.48it/s]
Labels shape:    4271
Frequent labels: 739
Few labels:      3369
Zero labels:     163


In [21]:
test=lmtc.load_dataset("test")

100%|██████████| 6000/6000 [05:21<00:00, 18.69it/s]


In [ ]:
DATA_SET_DIR="/mnt/localdata/geng/lib/lmtc-eurlex57k/data/datasets/"

MODELS_DIR="/mnt/localdata/geng/lib/lmtc-eurlex57k/data/models/"